In [1]:

import numpy as np
import matplotlib.pyplot as plt
import librosa
import pyaudio
import threading
import queue
import time
from collections import deque
from scipy.fft import fft, fftfreq
from scipy import signal
import tkinter as tk
from tkinter import ttk
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


In [ ]:

from matplotlib import rcParams
from matplotlib import font_manager as fm

# Load font explicitly
devanagari_font_path = "/Users/sriniccaohri/Downloads/Noto_Sans_Devanagari/NotoSansDevanagari-VariableFont_wdth\,wght.ttf "  # Adjust path if different
devanagari_font = fm.FontProperties(fname=devanagari_font_path)


In [2]:
# Real-Time Music Transcriber with Hindustani (North Indian Classical) Music Support
# Supports both Western and Hindustani note systems

class RealTimeMusicTranscriber:
    def __init__(self):
        # Audio settings
        self.CHUNK = 4096  # Audio buffer size
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1  # Mono audio
        self.RATE = 22050  # Sample rate
        
        # Processing settings
        self.BUFFER_DURATION = 2.0  # seconds of audio to keep in buffer
        self.ANALYSIS_INTERVAL = 0.1  # How often to analyze (seconds)
        self.MIN_CONFIDENCE = 0.5  # Minimum confidence for note detection
        
        # Note system selection
        self.current_system = "western"  # default to western
        
        # Initialize PyAudio
        self.audio = pyaudio.PyAudio()
        self.stream = None
        
        # Audio buffer - stores recent audio data
        buffer_size = int(self.BUFFER_DURATION * self.RATE)
        self.audio_buffer = deque(maxlen=buffer_size)
        
        # Results storage
        self.detected_notes = deque(maxlen=100)  # Last 100 detections
        self.timestamps = deque(maxlen=100)
        self.frequencies = deque(maxlen=100)
        self.confidences = deque(maxlen=100)
        
        # Threading controls
        self.is_recording = False
        self.is_analyzing = False
        self.audio_queue = queue.Queue()
        
        # Musical note frequencies (both systems)
        self.setup_note_frequencies()
        
        # GUI components
        self.root = None
        self.canvas = None
        self.fig = None
        
    def setup_note_frequencies(self):
        """Setup musical note frequency mappings for both Western and Hindustani systems"""
        
        # Western notes (12-tone equal temperament)
        self.western_frequencies = {
            'C3': 130.81, 'C#3': 138.59, 'D3': 146.83, 'D#3': 155.56, 'E3': 164.81,
            'F3': 174.61, 'F#3': 185.00, 'G3': 196.00, 'G#3': 207.65, 'A3': 220.00,
            'A#3': 233.08, 'B3': 246.94,
            'C4': 261.63, 'C#4': 277.18, 'D4': 293.66, 'D#4': 311.13, 'E4': 329.63,
            'F4': 349.23, 'F#4': 369.99, 'G4': 392.00, 'G#4': 415.30, 'A4': 440.00,
            'A#4': 466.16, 'B4': 493.88,
            'C5': 523.25, 'C#5': 554.37, 'D5': 587.33, 'D#5': 622.25, 'E5': 659.25,
            'F5': 698.46, 'F#5': 739.99, 'G5': 783.99, 'G#5': 830.61, 'A5': 880.00,
            'A#5': 932.33, 'B5': 987.77,
            'C6': 1046.50
        }
        
        # Hindustani notes (Sargam system) - frequencies based on just intonation
        # Standard tuning with Sa = C4 (261.63 Hz)
        sa_freq = 261.63  # Sa (C4)
        
        # Just intonation ratios for Hindustani music
        ratios = {
            'Sa': 1.0,           # Shadja (Tonic)
            'Re_k': 16/15,       # Komal Rishabh (minor 2nd)
            'Re': 9/8,           # Shuddh Rishabh (major 2nd)
            'Ga_k': 6/5,         # Komal Gandhar (minor 3rd)
            'Ga': 5/4,           # Shuddh Gandhar (major 3rd)
            'Ma': 4/3,           # Shuddh Madhyam (perfect 4th)
            'Ma_t': 45/32,       # Tivra Madhyam (augmented 4th)
            'Pa': 3/2,           # Pancham (perfect 5th)
            'Dha_k': 8/5,        # Komal Dhaivat (minor 6th)
            'Dha': 5/3,          # Shuddh Dhaivat (major 6th)
            'Ni_k': 16/9,        # Komal Nishad (minor 7th)
            'Ni': 15/8,          # Shuddh Nishad (major 7th)
        }
        
        # Generate Hindustani frequencies across octaves
        self.hindustani_frequencies = {}
        
        # Generate for octaves 3, 4, 5, 6
        for octave in [3, 4, 5, 6]:
            octave_multiplier = 2 ** (octave - 4)  # Relative to octave 4
            base_freq = sa_freq * octave_multiplier
            
            for note, ratio in ratios.items():
                freq = base_freq * ratio
                note_name = f"{note}{octave}"
                self.hindustani_frequencies[note_name] = freq
        
        # Create display names for Hindustani notes
        self.hindustani_display_names = {
            'Sa': 'सा', 'Re_k': 'रे♭', 'Re': 'रे', 'Ga_k': 'ग♭', 'Ga': 'ग',
            'Ma': 'म', 'Ma_t': 'म♯', 'Pa': 'प', 'Dha_k': 'ध♭', 'Dha': 'ध',
            'Ni_k': 'नि♭', 'Ni': 'नि'
        }
        
        # Set current system
        self.update_current_system("western")
    
    def update_current_system(self, system):
        """Update the current note system"""
        self.current_system = system
        
        if system == "western":
            self.note_frequencies = self.western_frequencies.copy()
            self.note_names = list(self.western_frequencies.keys())
            self.frequencies_list = list(self.western_frequencies.values())
        else:  # hindustani
            self.note_frequencies = self.hindustani_frequencies.copy()
            self.note_names = list(self.hindustani_frequencies.keys())
            self.frequencies_list = list(self.hindustani_frequencies.values())
    
    def get_display_name(self, note):
        """Get display name for a note (Devanagari for Hindustani)"""
        if self.current_system == "hindustani" and note:
            # Extract base note name (without octave)
            base_note = note[:-1]  # Remove octave number
            octave = note[-1]      # Get octave number
            
            if base_note in self.hindustani_display_names:
                return f"{self.hindustani_display_names[base_note]}{octave}"
        
        return note

    def audio_callback(self, in_data, frame_count, time_info, status):
        """Callback function for PyAudio stream"""
        if status:
            print(f"Audio callback status: {status}")
        
        # Convert bytes to numpy array
        audio_data = np.frombuffer(in_data, dtype=np.float32)
        
        # Add to queue for processing
        try:
            self.audio_queue.put_nowait(audio_data)
        except queue.Full:
            # Queue is full, skip this chunk
            pass
            
        return (None, pyaudio.paContinue)

    def start_audio_stream(self):
        """Start the microphone audio stream"""
        try:
            self.stream = self.audio.open(
                format=self.FORMAT,
                channels=self.CHANNELS,
                rate=self.RATE,
                input=True,
                frames_per_buffer=self.CHUNK,
                stream_callback=self.audio_callback
            )
            
            self.stream.start_stream()
            self.is_recording = True
            print("Audio stream started successfully")
            return True
            
        except Exception as e:
            print(f"Error starting audio stream: {e}")
            return False

    def stop_audio_stream(self):
        """Stop the microphone audio stream"""
        self.is_recording = False
        
        if self.stream:
            self.stream.stop_stream()
            self.stream.close()
            self.stream = None
            
        print("Audio stream stopped")

    def audio_buffer_thread(self):
        """Thread to manage audio buffer from queue"""
        while self.is_recording:
            try:
                # Get audio data from queue (with timeout)
                audio_chunk = self.audio_queue.get(timeout=0.1)
                
                # Add to circular buffer
                self.audio_buffer.extend(audio_chunk)
                
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Audio buffer thread error: {e}")

    def detect_pitch_realtime(self, audio_segment):
        """Optimized pitch detection for real-time use"""
        if len(audio_segment) < 512:
            return 0, 0
            
        # Quick volume check
        if np.max(np.abs(audio_segment)) < 0.01:
            return 0, 0
        
        # Apply window and FFT
        windowed = audio_segment * np.hanning(len(audio_segment))
        fft_result = np.abs(fft(windowed))
        freqs = fftfreq(len(windowed), 1/self.RATE)
        
        # Focus on positive frequencies in vocal range (80 Hz - 2000 Hz)
        positive_freqs = freqs[:len(freqs)//2]
        positive_fft = fft_result[:len(fft_result)//2]
        
        # Filter to vocal range
        vocal_mask = (positive_freqs >= 80) & (positive_freqs <= 2000)
        vocal_freqs = positive_freqs[vocal_mask]
        vocal_fft = positive_fft[vocal_mask]
        
        if len(vocal_fft) == 0:
            return 0, 0
        
        # Find peaks
        threshold = np.max(vocal_fft) * 0.2
        peaks, properties = signal.find_peaks(vocal_fft, height=threshold, distance=5)
        
        if len(peaks) > 0:
            # Get the strongest peak
            strongest_peak_idx = peaks[np.argmax(vocal_fft[peaks])]
            dominant_freq = vocal_freqs[strongest_peak_idx]
            confidence = vocal_fft[strongest_peak_idx] / np.max(vocal_fft)
            return dominant_freq, confidence
            
        return 0, 0

    def frequency_to_note_realtime(self, frequency, tolerance=30):
        """Convert frequency to note with relaxed tolerance for real-time"""
        if frequency <= 0:
            return None, 0
        
        # Adjust tolerance for Hindustani system (more microtonal variations)
        if self.current_system == "hindustani":
            tolerance = 40  # Slightly higher tolerance for just intonation
            
        differences = [abs(frequency - freq) for freq in self.frequencies_list]
        min_diff_idx = np.argmin(differences)
        difference = differences[min_diff_idx]
        
        if difference <= tolerance:
            return self.note_names[min_diff_idx], 1 - (difference / tolerance)
        else:
            return None, 0

    def analysis_thread(self):
        """Thread for continuous audio analysis"""
        while self.is_analyzing:
            try:
                # Wait for enough audio data
                if len(self.audio_buffer) < self.RATE * 0.5:  # Need at least 0.5 seconds
                    time.sleep(0.05)
                    continue
                
                # Get recent audio for analysis
                analysis_length = int(self.RATE * 0.5)  # 0.5 seconds
                audio_segment = np.array(list(self.audio_buffer)[-analysis_length:])
                
                # Detect pitch
                frequency, confidence = self.detect_pitch_realtime(audio_segment)
                
                # Convert to note
                note, note_confidence = self.frequency_to_note_realtime(frequency)
                
                # Combined confidence
                total_confidence = confidence * note_confidence
                
                # Store results if confident enough
                current_time = time.time()
                
                if total_confidence >= self.MIN_CONFIDENCE and note:
                    self.detected_notes.append(note)
                    self.timestamps.append(current_time)
                    self.frequencies.append(frequency)
                    self.confidences.append(total_confidence)
                else:
                    # Still store for visualization, but mark as uncertain
                    self.detected_notes.append(None)
                    self.timestamps.append(current_time)
                    self.frequencies.append(frequency)
                    self.confidences.append(total_confidence)
                
                # Control analysis rate
                time.sleep(self.ANALYSIS_INTERVAL)
                
            except Exception as e:
                print(f"Analysis thread error: {e}")
                time.sleep(0.1)

    def on_system_change(self):
        """Handle note system change"""
        new_system = self.system_var.get()
        self.update_current_system(new_system)
        
        # Clear previous detections to avoid confusion
        self.detected_notes.clear()
        self.timestamps.clear()
        self.frequencies.clear()
        self.confidences.clear()
        
        print(f"Switched to {new_system} note system")

    def create_gui(self):
        """Create the GUI for real-time visualization"""
        self.root = tk.Tk()
        self.root.title("Real-Time Music Transcriber - Western & Hindustani")
        self.root.geometry("900x700")
        
        # Control frame
        control_frame = ttk.Frame(self.root)
        control_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Buttons
        self.start_button = ttk.Button(control_frame, text="Start Recording", 
                                      command=self.start_transcription)
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.stop_button = ttk.Button(control_frame, text="Stop Recording", 
                                     command=self.stop_transcription, state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # Note system selection
        system_frame = ttk.LabelFrame(control_frame, text="Note System")
        system_frame.pack(side=tk.LEFT, padx=20)
        
        self.system_var = tk.StringVar(value="western")
        western_radio = ttk.Radiobutton(system_frame, text="Western (Do-Re-Mi)", 
                                       variable=self.system_var, value="western",
                                       command=self.on_system_change)
        western_radio.pack(side=tk.LEFT, padx=5)
        
        hindustani_radio = ttk.Radiobutton(system_frame, text="Hindustani (सा-रे-ग)", 
                                          variable=self.system_var, value="hindustani",
                                          command=self.on_system_change)
        hindustani_radio.pack(side=tk.LEFT, padx=5)
        
        # Status label
        self.status_label = ttk.Label(control_frame, text="Ready to start")
        self.status_label.pack(side=tk.LEFT, padx=20)
        
        # Current note display
        note_frame = ttk.Frame(control_frame)
        note_frame.pack(side=tk.RIGHT, padx=5)
        
        ttk.Label(note_frame, text="Current Note:").pack()
        self.note_label = ttk.Label(note_frame, text="--", 
                                   font=("Arial", 16, "bold"),
                                   foreground="red")
        self.note_label.pack()
        
        # Info frame
        info_frame = ttk.Frame(self.root)
        info_frame.pack(fill=tk.X, padx=5, pady=2)
        
        info_text = ("Western: C D E F G A B | Hindustani: सा रे ग म प ध नि | " +
                    "♭=komal(flat) ♯=tivra(sharp)")
        ttk.Label(info_frame, text=info_text, font=("Arial", 9)).pack()
        
        # Create matplotlib figure
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(12, 7))
        self.fig.tight_layout(pad=3.0)
        
        # Embed matplotlib in tkinter
        self.canvas = FigureCanvasTkAgg(self.fig, self.root)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Setup plots
        self.setup_plots()
        
        # Animation for real-time updates
        self.ani = animation.FuncAnimation(self.fig, self.update_plots, 
                                          interval=100, blit=False)

    def setup_plots(self):
        """Setup the matplotlib plots"""
        # Frequency plot
        self.ax1.set_title("Real-Time Frequency Detection")
        self.ax1.set_ylabel("Frequency (Hz)")
        self.ax1.set_ylim(100, 1000)
        self.ax1.grid(True, alpha=0.3)
        
        # Notes plot
        self.ax2.set_title("Detected Musical Notes")
        self.ax2.set_ylabel("Note")
        self.ax2.set_xlabel("Time (seconds ago)")
        self.ax2.grid(True, alpha=0.3)

    def update_plots(self, frame):
        """Update plots with real-time data"""
        if not self.is_analyzing or len(self.timestamps) == 0:
            return
        
        # Convert timestamps to relative time
        if len(self.timestamps) > 0:
            current_time = time.time()
            relative_times = [current_time - t for t in reversed(list(self.timestamps))]
            relative_times.reverse()
            
            # Get recent data (last 30 seconds)
            recent_mask = [t <= 30 for t in relative_times]
            recent_times = [t for t, m in zip(relative_times, recent_mask) if m]
            recent_freqs = [f for f, m in zip(list(self.frequencies), recent_mask) if m]
            recent_notes = [n for n, m in zip(list(self.detected_notes), recent_mask) if m]
            recent_conf = [c for c, m in zip(list(self.confidences), recent_mask) if m]
            
            if recent_times:
                # Update frequency plot
                self.ax1.clear()
                self.ax1.plot(recent_times, recent_freqs, 'b-', alpha=0.7)
                self.ax1.set_title(f"Real-Time Frequency Detection ({self.current_system.title()} System)")
                self.ax1.set_ylabel("Frequency (Hz)")
                self.ax1.set_ylim(100, 1000)
                self.ax1.grid(True, alpha=0.3)
                
                # Update notes plot
                self.ax2.clear()
                valid_times = []
                valid_note_indices = []
                
                for t, note, conf in zip(recent_times, recent_notes, recent_conf):
                    if note and conf >= self.MIN_CONFIDENCE:
                        valid_times.append(t)
                        if note in self.note_names:
                            valid_note_indices.append(self.note_names.index(note))
                
                if valid_times:
                    self.ax2.scatter(valid_times, valid_note_indices, c='red', s=50, alpha=0.7)
                    
                    # Set y-axis to show note names
                    if valid_note_indices:
                        min_idx = max(0, min(valid_note_indices) - 2)
                        max_idx = min(len(self.note_names) - 1, max(valid_note_indices) + 2)
                        note_range = range(min_idx, max_idx + 1)
                        self.ax2.set_yticks(note_range)
                        
                        # Use display names (with Devanagari for Hindustani)
                        display_labels = []
                        for i in note_range:
                            if i < len(self.note_names):
                                display_name = self.get_display_name(self.note_names[i])
                                display_labels.append(display_name)
                        
                        self.ax2.set_yticklabels(display_labels)
                
                system_title = "Western Musical Notes" if self.current_system == "western" else "Hindustani Musical Notes (सरगम)"
                self.ax2.set_title(f"Detected {system_title}")
                self.ax2.set_ylabel("Note")
                self.ax2.set_xlabel("Time (seconds ago)")
                self.ax2.grid(True, alpha=0.3)
                
                # Update current note display
                if recent_notes and recent_conf:
                    latest_note = recent_notes[-1]
                    latest_conf = recent_conf[-1]
                    
                    if latest_note and latest_conf >= self.MIN_CONFIDENCE:
                        display_note = self.get_display_name(latest_note)
                        self.note_label.config(text=display_note)
                    else:
                        self.note_label.config(text="--")

    def start_transcription(self):
        """Start real-time transcription"""
        if self.start_audio_stream():
            self.is_analyzing = True
            
            # Start threads
            self.buffer_thread = threading.Thread(target=self.audio_buffer_thread)
            self.buffer_thread.daemon = True
            self.buffer_thread.start()
            
            self.analysis_thread_obj = threading.Thread(target=self.analysis_thread)
            self.analysis_thread_obj.daemon = True
            self.analysis_thread_obj.start()
            
            # Update GUI
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.status_label.config(text=f"Recording ({self.current_system})...")
            
            print(f"Real-time transcription started in {self.current_system} mode!")
        else:
            self.status_label.config(text="Error: Could not start audio")

    def stop_transcription(self):
        """Stop real-time transcription"""
        self.is_analyzing = False
        self.stop_audio_stream()
        
        # Update GUI
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_label.config(text="Stopped")
        self.note_label.config(text="--")
        
        print("Real-time transcription stopped!")

    def export_session(self, filename=None):
        """Export the current session data"""
        if not self.detected_notes:
            print("No data to export")
            return
        
        if filename is None:
            filename = f"session_{self.current_system}_{int(time.time())}.txt"
            
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"Real-Time Music Transcription Session\n")
            f.write(f"Note System: {self.current_system.title()}\n")
            f.write(f"Date: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("-" * 50 + "\n")
            f.write("Time(s)\tNote\tDisplay\tFrequency(Hz)\tConfidence\n")
            
            base_time = min(self.timestamps) if self.timestamps else 0
            
            for timestamp, note, freq, conf in zip(self.timestamps, self.detected_notes, 
                                                  self.frequencies, self.confidences):
                relative_time = timestamp - base_time
                note_str = note if note else "None"
                display_str = self.get_display_name(note) if note else "None"
                f.write(f"{relative_time:.3f}\t{note_str}\t{display_str}\t{freq:.1f}\t{conf:.3f}\n")
        
        print(f"Session exported to {filename}")

    def cleanup(self):
        """Clean up resources"""
        self.stop_transcription()
        if self.audio:
            self.audio.terminate()

    def run(self):
        """Run the real-time transcriber"""
        try:
            self.create_gui()
            
            # Handle window closing
            def on_closing():
                self.cleanup()
                self.root.destroy()
            
            self.root.protocol("WM_DELETE_WINDOW", on_closing)
            
            # Start the GUI
            print("Starting Real-Time Music Transcriber with Hindustani Support...")
            print("Choose between Western and Hindustani note systems!")
            print("Click 'Start Recording' to begin!")
            self.root.mainloop()
            
        except KeyboardInterrupt:
            print("\nShutting down...")
            self.cleanup()
        except Exception as e:
            print(f"Error: {e}")
            self.cleanup()

In [3]:
# Simplified console version for testing both systems
class SimpleRealTimeTranscriber:
    """Simplified version that prints notes to console with system choice"""
    
    def __init__(self, system="western"):
        self.transcriber = RealTimeMusicTranscriber()
        self.transcriber.update_current_system(system)
        
    def run_console(self, duration=30):
        """Run transcription for specified duration and print to console"""
        print(f"Starting {duration}-second real-time transcription...")
        print(f"Using {self.transcriber.current_system} note system")
        print("Start humming or singing!")
        
        if not self.transcriber.start_audio_stream():
            print("Failed to start audio stream")
            return
        
        self.transcriber.is_analyzing = True
        
        # Start threads
        buffer_thread = threading.Thread(target=self.transcriber.audio_buffer_thread)
        buffer_thread.daemon = True
        buffer_thread.start()
        
        analysis_thread = threading.Thread(target=self.transcriber.analysis_thread)
        analysis_thread.daemon = True
        analysis_thread.start()
        
        # Monitor for specified duration
        start_time = time.time()
        last_note = None
        
        try:
            while time.time() - start_time < duration:
                if len(self.transcriber.detected_notes) > 0:
                    current_note = list(self.transcriber.detected_notes)[-1]
                    current_conf = list(self.transcriber.confidences)[-1]
                    
                    if (current_note != last_note and current_note and 
                        current_conf >= self.transcriber.MIN_CONFIDENCE):
                        display_note = self.transcriber.get_display_name(current_note)
                        print(f"♪ {display_note} (confidence: {current_conf:.2f})")
                        last_note = current_note
                
                time.sleep(0.1)
                
        except KeyboardInterrupt:
            print("\nStopped by user")
        
        # Stop transcription
        self.transcriber.stop_transcription()
        
        # Export results
        self.transcriber.export_session()
        print("Session saved!")

In [4]:
# Installation and setup instructions
def print_setup_instructions():
    """Print setup instructions for users"""
    print("""
    REAL-TIME MUSIC TRANSCRIBER WITH HINDUSTANI SUPPORT
    ================================================
    
    🎵 FEATURES:
    - Western notes (C, D, E, F, G, A, B) with sharps/flats
    - Hindustani notes (सा, रे, ग, म, प, ध, नि) with komal/tivra variants
    - Real-time GUI with switchable note systems
    - Devanagari script display for Indian classical notes
    - Export functionality for both systems
    
    🎼 HINDUSTANI NOTE SYSTEM:
    सा (Sa) = C, रे (Re) = D, ग (Ga) = E, म (Ma) = F
    प (Pa) = G, ध (Dha) = A, नि (Ni) = B
    
    ♭ = komal (flat), ♯ = tivra (sharp)
    
    🚀 USAGE:
    
    # Full GUI with both systems:
    transcriber = RealTimeMusicTranscriber()
    transcriber.run()
    
    # Console version - Western:
    simple = SimpleRealTimeTranscriber("western")
    simple.run_console(30)
    
    # Console version - Hindustani:  
    simple = SimpleRealTimeTranscriber("hindustani")
    simple.run_console(30)
    
    🎯 TIPS FOR BEST RESULTS:
    - Use a good quality microphone
    - Sing/hum clearly and steadily
    - Minimize background noise
    - For Hindustani: Try traditional ragas and meends
    - Adjust confidence threshold if needed
    
    
    """)

In [ ]:
if __name__ == "__main__":
    print_setup_instructions()
    
    print("\nChoose an option:")
    print("1. Run GUI version (recommended)")
    print("2. Run console version (simple)")
    print("3. Just show setup instructions")
    
    choice = input("Enter choice (1-3): ").strip()
    
    if choice == "1":
        try:
            transcriber = RealTimeMusicTranscriber()
            transcriber.run()
        except ImportError as e:
            print(f"Missing required package: {e}")
            print("Please install required packages first (see instructions above)")
    
    elif choice == "2":
        try:
            simple = SimpleRealTimeTranscriber()
            duration = input("Recording duration in seconds (default 30): ").strip()
            duration = int(duration) if duration.isdigit() else 30
            simple.run_console(duration)
        except ImportError as e:
            print(f"Missing required package: {e}")
            print("Please install required packages first (see instructions above)")
    
    else:
        print("Setup instructions displayed above. Install the packages and run again!")


    REAL-TIME MUSIC TRANSCRIBER WITH HINDUSTANI SUPPORT
    
    🎵 FEATURES:
    - Western notes (C, D, E, F, G, A, B) with sharps/flats
    - Hindustani notes (सा, रे, ग, म, प, ध, नि) with komal/tivra variants
    - Real-time GUI with switchable note systems
    - Devanagari script display for Indian classical notes
    - Export functionality for both systems
    
    🎼 HINDUSTANI NOTE SYSTEM:
    सा (Sa) = C, रे (Re) = D, ग (Ga) = E, म (Ma) = F
    प (Pa) = G, ध (Dha) = A, नि (Ni) = B
    
    ♭ = komal (flat), ♯ = tivra (sharp)
    
    🚀 USAGE:
    
    # Full GUI with both systems:
    transcriber = RealTimeMusicTranscriber()
    transcriber.run()
    
    # Console version - Western:
    simple = SimpleRealTimeTranscriber("western")
    simple.run_console(30)
    
    # Console version - Hindustani:  
    simple = SimpleRealTimeTranscriber("hindustani")
    simple.run_console(30)
    
    🎯 TIPS FOR BEST RESULTS:
    - Use a good quality microphone
    - Sing/hum clearly and ste

Enter choice (1-3):  1


/var/folders/4_/tvb0w_9s2gsb1r0z05xx2lg00000gn/T/ipykernel_71159/4189594136.py:378: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  self.ani = animation.FuncAnimation(self.fig, self.update_plots,


Starting Real-Time Music Transcriber with Hindustani Support...
Choose between Western and Hindustani note systems!
Click 'Start Recording' to begin!


2025-06-02 20:40:16.021 Python[71159:4899223] +[IMKClient subclass]: chose IMKClient_Modern
2025-06-02 20:40:16.022 Python[71159:4899223] +[IMKInputSession subclass]: chose IMKInputSession_Modern


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2352 (\N{DEVANAGARI LETTER RA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Matplotlib currently does not support Devanagari natively.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2375 (\N{DEVANAGARI VOWEL SIGN E}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2327 (\N{DEVANAGARI LETTER GA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2350 (\N{DEVANAGARI LETTER MA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/pytho

Switched to western note system
Audio stream stopped
Real-time transcription stopped!
Audio stream started successfully
Real-time transcription started in western mode!
Switched to hindustani note system


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2343 (\N{DEVANAGARI LETTER DHA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Matplotlib currently does not support Devanagari natively.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2344 (\N{DEVANAGARI LETTER NA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2367 (\N{DEVANAGARI VOWEL SIGN I}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2346 (\N{DEVANAGARI LETTER PA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/pyth

Switched to western note system


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2344 (\N{DEVANAGARI LETTER NA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Matplotlib currently does not support Devanagari natively.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2367 (\N{DEVANAGARI VOWEL SIGN I}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2360 (\N{DEVANAGARI LETTER SA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py:862: UserWarning: Glyph 2366 (\N{DEVANAGARI VOWEL SIGN AA}) missing from font(s) DejaVu Sans.
  func(*args)
/Library/Frameworks/Python.framework/Versions/3.12/lib/p